# Tutoriel sur l'Algorithme EM et l'Apprentissage par Renforcement

Ce notebook présente deux concepts fondamentaux en apprentissage automatique :
1. L'algorithme d'Expectation-Maximization (EM)
2. L'apprentissage par renforcement (Reinforcement Learning)

Nous utiliserons principalement NumPy pour implémenter ces algorithmes et générer des données pour nos expériences.

## Partie 1: Algorithme d'Expectation-Maximization (EM)

L'algorithme EM est une méthode itérative pour trouver les estimations du maximum de vraisemblance des paramètres dans des modèles statistiques, où le modèle dépend de variables latentes non observées.

### Application: Mélange de Gaussiennes (Gaussian Mixture Model)

L'un des cas d'utilisation les plus courants de l'algorithme EM est l'estimation des paramètres d'un mélange de distributions gaussiennes.

In [ ]:
# Importation des bibliothèques nécessaires
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import multivariate_normal
from sklearn.datasets import make_blobs

# Configurer matplotlib pour des graphiques plus agréables
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)
np.random.seed(42)

### Génération de données pour le mélange de gaussiennes

Commençons par générer des données qui suivent un mélange de 3 distributions gaussiennes.

In [ ]:
# Paramètres pour la génération des données
n_samples = 1000
n_components = 3

# Les vraies moyennes de nos gaussiennes
true_means = np.array([[-2, -2], [0, 0], [2, 2]])
# Les vraies covariances
true_covs = np.array([
    [[0.5, 0], [0, 0.5]],
    [[1, 0], [0, 1]],
    [[0.5, 0], [0, 0.5]]
])
# Les vrais poids des composantes
true_weights = np.array([0.3, 0.4, 0.3])

# Générer les données
X, y = make_blobs(n_samples=n_samples, centers=true_means, 
                  cluster_std=np.sqrt(np.array([0.5, 1, 0.5])),
                  random_state=42)

# Appliquer les poids (rééchantillonner pour respecter les proportions)
indices = np.random.choice(np.arange(n_samples), size=n_samples, 
                         p=np.bincount(y) / n_samples)
X = X[indices]
y = y[indices]

# Visualiser les données générées
plt.figure(figsize=(10, 6))
plt.scatter(X[:, 0], X[:, 1], c=y, s=30, cmap='viridis', alpha=0.7)
plt.title('Données générées: Mélange de 3 gaussiennes')
plt.xlabel('Feature 1')
plt.ylabel('Feature 2')
plt.colorbar(label='Cluster')
plt.grid(True)
plt.show()

### Implémentation de l'algorithme EM pour le mélange de gaussiennes

L'algorithme EM consiste en deux étapes principales qui sont répétées jusqu'à convergence :
1. **Étape E (Expectation)** : Calculer les probabilités postérieures (responsabilités) de chaque composante pour chaque point de données
2. **Étape M (Maximization)** : Mettre à jour les paramètres (moyennes, covariances, poids) en utilisant ces responsabilités

In [ ]:
class GaussianMixtureEM:
    def __init__(self, n_components=3, max_iter=100, tol=1e-6, random_state=None):
        self.n_components = n_components
        self.max_iter = max_iter
        self.tol = tol
        self.random_state = random_state
        np.random.seed(random_state)
        
    def initialize_parameters(self, X):
        n_samples, n_features = X.shape
        
        # Initialiser les moyennes en sélectionnant aléatoirement des points de données
        indices = np.random.choice(n_samples, self.n_components, replace=False)
        self.means = X[indices]
        
        # Initialiser les covariances à l'identité
        self.covariances = np.array([np.eye(n_features) for _ in range(self.n_components)])
        
        # Initialiser les poids uniformément
        self.weights = np.ones(self.n_components) / self.n_components
        
    def e_step(self, X):
        """Étape E: Calcul des responsabilités"""
        n_samples = X.shape[0]
        responsibilities = np.zeros((n_samples, self.n_components))
        
        # Calculer la densité de probabilité pour chaque point et chaque composante
        for k in range(self.n_components):
            responsibilities[:, k] = self.weights[k] * multivariate_normal.pdf(
                X, mean=self.means[k], cov=self.covariances[k])
        
        # Normaliser pour obtenir les probabilités postérieures
        responsibilities_sum = responsibilities.sum(axis=1, keepdims=True)
        responsibilities /= responsibilities_sum
        
        return responsibilities
    
    def m_step(self, X, responsibilities):
        """Étape M: Mise à jour des paramètres"""
        n_samples = X.shape[0]
        
        # Calculer les responsabilités totales pour chaque composante
        resp_sum = responsibilities.sum(axis=0)
        
        # Mettre à jour les poids
        self.weights = resp_sum / n_samples
        
        # Mettre à jour les moyennes
        self.means = np.dot(responsibilities.T, X) / resp_sum[:, np.newaxis]
        
        # Mettre à jour les covariances
        for k in range(self.n_components):
            diff = X - self.means[k]
            self.covariances[k] = np.dot(responsibilities[:, k] * diff.T, diff) / resp_sum[k]
            
            # Ajouter une petite régularisation pour éviter les problèmes numériques
            self.covariances[k] += 1e-6 * np.eye(X.shape[1])
    
    def compute_log_likelihood(self, X):
        """Calculer la log-vraisemblance des données"""
        n_samples = X.shape[0]
        likelihood = np.zeros((n_samples, self.n_components))
        
        for k in range(self.n_components):
            likelihood[:, k] = self.weights[k] * multivariate_normal.pdf(
                X, mean=self.means[k], cov=self.covariances[k])
        
        return np.sum(np.log(np.sum(likelihood, axis=1)))
    
    def fit(self, X):
        """Ajuster le modèle aux données"""
        self.initialize_parameters(X)
        
        log_likelihood_history = []
        prev_log_likelihood = -np.inf
        
        for i in range(self.max_iter):
            # Étape E
            responsibilities = self.e_step(X)
            
            # Étape M
            self.m_step(X, responsibilities)
            
            # Calculer la log-vraisemblance
            log_likelihood = self.compute_log_likelihood(X)
            log_likelihood_history.append(log_likelihood)
            
            # Vérifier la convergence
            if abs(log_likelihood - prev_log_likelihood) < self.tol:
                break
                
            prev_log_likelihood = log_likelihood
            
            # Afficher les progrès tous les 5 itérations
            if (i + 1) % 5 == 0:
                print(f"Itération {i+1}/{self.max_iter}, Log-vraisemblance: {log_likelihood:.2f}")
        
        self.log_likelihood_history = log_likelihood_history
        self.n_iter_ = i + 1
        return self
    
    def predict(self, X):
        """Prédire les clusters pour les données X"""
        responsibilities = self.e_step(X)
        return np.argmax(responsibilities, axis=1)

### Ajustement du modèle et visualisation des résultats

In [ ]:
# Mélanger les données (pour oublier les vrais clusters)
np.random.shuffle(X)

# Ajuster le modèle EM
gmm = GaussianMixtureEM(n_components=3, max_iter=50, random_state=42)
gmm.fit(X)

# Prédire les clusters
y_pred = gmm.predict(X)

# Afficher la convergence de la log-vraisemblance
plt.figure(figsize=(10, 4))
plt.plot(gmm.log_likelihood_history)
plt.xlabel('Itérations')
plt.ylabel('Log-vraisemblance')
plt.title('Convergence de la log-vraisemblance')
plt.grid(True)
plt.show()

In [ ]:
# Visualiser les résultats
plt.figure(figsize=(12, 4))

# Données avec prédictions
plt.subplot(1, 2, 1)
plt.scatter(X[:, 0], X[:, 1], c=y_pred, s=30, cmap='viridis', alpha=0.7)
plt.title('Clusters détectés par EM')
plt.xlabel('Feature 1')
plt.ylabel('Feature 2')
plt.colorbar(label='Cluster')
plt.grid(True)

# Visualiser les gaussiennes estimées
plt.subplot(1, 2, 2)
plt.scatter(X[:, 0], X[:, 1], c=y_pred, s=30, cmap='viridis', alpha=0.2)

# Créer une grille pour visualiser les contours de densité
x_min, x_max = X[:, 0].min() - 1, X[:, 0].max() + 1
y_min, y_max = X[:, 1].min() - 1, X[:, 1].max() + 1
xx, yy = np.meshgrid(np.linspace(x_min, x_max, 100), np.linspace(y_min, y_max, 100))
grid = np.c_[xx.ravel(), yy.ravel()]

# Calculer la densité pour chaque point de la grille
Z = np.zeros((grid.shape[0], gmm.n_components))
for k in range(gmm.n_components):
    Z[:, k] = gmm.weights[k] * multivariate_normal.pdf(
        grid, mean=gmm.means[k], cov=gmm.covariances[k])
Z = Z.sum(axis=1).reshape(xx.shape)

# Tracer les contours de densité
plt.contour(xx, yy, Z, levels=5, cmap='viridis')

# Tracer les centres des gaussiennes
plt.scatter(gmm.means[:, 0], gmm.means[:, 1], c='red', s=200, marker='x')

plt.title('Gaussiennes estimées')
plt.xlabel('Feature 1')
plt.ylabel('Feature 2')
plt.grid(True)
plt.tight_layout()
plt.show()

### Comparer avec les vrais paramètres

In [ ]:
print("=== Comparaison des paramètres estimés vs réels ===")
print("\nMoyennes estimées:")
print(gmm.means)
print("\nVraies moyennes:")
print(true_means)

print("\nCovariances estimées:")
for i, cov in enumerate(gmm.covariances):
    print(f"Composante {i+1}:\n{cov}")
print("\nVraies covariances:")
for i, cov in enumerate(true_covs):
    print(f"Composante {i+1}:\n{cov}")

print("\nPoids estimés:")
print(gmm.weights)
print("\nVrais poids:")
print(true_weights)

### Exemple avancé: EM pour un mélange bimodal en 1D

Illustrons encore le fonctionnement de l'algorithme EM pour un mélange de deux gaussiennes en une dimension. Cela nous permettra de mieux visualiser les étapes de l'algorithme.

In [ ]:
# Générer des données 1D suivant un mélange de deux gaussiennes
np.random.seed(42)
n1, n2 = 300, 700  # Nombre d'échantillons par gaussienne
mu1, sigma1 = -2, 0.5  # Moyenne et écart-type de la première gaussienne
mu2, sigma2 = 2, 1.0   # Moyenne et écart-type de la deuxième gaussienne

# Générer les échantillons
samples1 = np.random.normal(mu1, sigma1, n1)
samples2 = np.random.normal(mu2, sigma2, n2)
samples = np.concatenate([samples1, samples2])
np.random.shuffle(samples)  # Mélanger les échantillons

# Convertir en array 2D pour notre algorithme
X_1d = samples.reshape(-1, 1)

# Visualiser l'histogramme des données
plt.figure(figsize=(12, 6))
plt.hist(samples, bins=30, density=True, alpha=0.7, color='skyblue')
plt.title('Distribution des données: mélange de deux gaussiennes')
plt.xlabel('x')
plt.ylabel('Densité')

# Tracer les vraies densités
x = np.linspace(-6, 6, 1000)
true_density1 = n1 / (n1 + n2) * (1 / (sigma1 * np.sqrt(2 * np.pi))) * np.exp(-(x - mu1)**2 / (2 * sigma1**2))
true_density2 = n2 / (n1 + n2) * (1 / (sigma2 * np.sqrt(2 * np.pi))) * np.exp(-(x - mu2)**2 / (2 * sigma2**2))
plt.plot(x, true_density1, 'r-', lw=2, label=f'Gaussienne 1: μ={mu1}, σ={sigma1}')
plt.plot(x, true_density2, 'g-', lw=2, label=f'Gaussienne 2: μ={mu2}, σ={sigma2}')
plt.plot(x, true_density1 + true_density2, 'k--', lw=2, label='Mélange total')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# Ajuster notre modèle EM au mélange 1D
gmm_1d = GaussianMixtureEM(n_components=2, max_iter=30, random_state=42)
gmm_1d.fit(X_1d)

# Récupérer les paramètres estimés
estimated_weights = gmm_1d.weights
estimated_means = gmm_1d.means.flatten()
estimated_stds = np.sqrt(gmm_1d.covariances.flatten())
estimated_cluster = gmm_1d.predict(X_1d)

# Afficher les résultats
print("=== Paramètres estimés ===")
print(f"Poids: {estimated_weights}")
print(f"Moyennes: {estimated_means}")
print(f"Écarts-types: {estimated_stds}")

print("\n=== Vrais paramètres ===")
print(f"Poids: [{n1/(n1+n2):.3f}, {n2/(n1+n2):.3f}]")
print(f"Moyennes: [{mu1}, {mu2}]")
print(f"Écarts-types: [{sigma1}, {sigma2}]")

In [ ]:
# Visualiser les résultats
plt.figure(figsize=(12, 6))
plt.hist(samples, bins=30, density=True, alpha=0.5, color='skyblue')

# Tracer les densités estimées
x = np.linspace(-6, 6, 1000)
est_density1 = estimated_weights[0] * (1 / (estimated_stds[0] * np.sqrt(2 * np.pi))) * np.exp(-(x - estimated_means[0])**2 / (2 * estimated_stds[0]**2))
est_density2 = estimated_weights[1] * (1 / (estimated_stds[1] * np.sqrt(2 * np.pi))) * np.exp(-(x - estimated_means[1])**2 / (2 * estimated_stds[1]**2))

plt.plot(x, est_density1, 'r-', lw=2, 
         label=f'Gaussienne estimée 1: π={estimated_weights[0]:.2f}, μ={estimated_means[0]:.2f}, σ={estimated_stds[0]:.2f}')
plt.plot(x, est_density2, 'g-', lw=2, 
         label=f'Gaussienne estimée 2: π={estimated_weights[1]:.2f}, μ={estimated_means[1]:.2f}, σ={estimated_stds[1]:.2f}')
plt.plot(x, est_density1 + est_density2, 'k--', lw=2, label='Mélange estimé total')

plt.title('Résultats de l\'algorithme EM: Mélange de gaussiennes en 1D')
plt.xlabel('x')
plt.ylabel('Densité')
plt.legend()
plt.grid(True)
plt.show()